limpiar col names Sheet

In [41]:
import pandas as pd
import glob
import re
import os

Separate Sheet

Generate simple dataset flujograma 

In [46]:
def simple_dataset_flujograma(result_df_list, file_path):
    """Genera dataset simplificado extrayendo intersección y tipo de día del filename"""
    import os
    
    # Extraer información del nombre del archivo
    basename = os.path.basename(file_path)
    name_without_ext = os.path.splitext(basename)[0]
    
    codigo = ''
    interseccion = ''
    
    if '_' in name_without_ext:
        parts = name_without_ext.split('_')
        
        # Si el primer elemento parece un código (números o letras+números cortos)
        first_part = parts[0]
        if len(first_part) <= 6 and (first_part.isdigit() or first_part.isalnum()):
            # Formato tipo: 58_SS77-01_Av. Rosa Lozano...
            codigo = first_part
            if len(parts) > 2:
                interseccion = '_'.join(parts[1:])  # Incluir segunda parte del código
            else:
                interseccion = '_'.join(parts[1:])
        else:
            # Formato tipo: SP01 Av. Los Héroes - Ca. Los Conquistadores_Día_Tipico
            # El código está al inicio sin separador
            codigo_match = re.match(r'^([A-Z]{2,4}\d{1,3})', name_without_ext)
            if codigo_match:
                codigo = codigo_match.group(1)
                interseccion = name_without_ext[len(codigo):].strip()
            else:
                codigo = parts[0] if len(parts) > 0 else ''
                interseccion = '_'.join(parts[1:]) if len(parts) > 1 else name_without_ext
    else:
        # Sin separadores de guión bajo
        codigo_match = re.match(r'^([A-Z]{2,4}\d{1,3})', name_without_ext)
        if codigo_match:
            codigo = codigo_match.group(1)
            interseccion = name_without_ext[len(codigo):].strip()
        else:
            interseccion = name_without_ext
    
    # Verificar si es atípico
    es_atipico = any(word in name_without_ext.lower() for word in ['atípico', 'atipico'])
    dia_tipo = 'Atípico' if es_atipico else 'Típico'
    
    # Limpiar intersección (remover referencias a día)
    palabras_remover = ['día_atípico', 'dia_atipico', 'día_típico', 'dia_tipico', 
                       '_día_', '_dia_', 'día', 'dia']
    
    for palabra in palabras_remover:
        interseccion = interseccion.replace(palabra, '').replace(palabra.capitalize(), '')
    
    # Limpiar puntos extras, guiones bajos y espacios múltiples
    interseccion = interseccion.replace('..', '').replace('_', ' ').strip()
    interseccion = re.sub(r'\s+', ' ', interseccion)
    interseccion = interseccion.strip()
    
    print(f"Archivo parseado: {basename}")
    print(f"  Código: '{codigo}'")
    print(f"  Intersección: '{interseccion}'")
    print(f"  Es atípico: {es_atipico}")
    print(f"  Día tipo: {dia_tipo}")
    
    all_data = []
    
    for df in result_df_list:
        # Crear dataset básico
        simple_data = {}
        
        # Datos del DataFrame
        #simple_data['hora'] = df['hora']
        simple_data['suma_vehiculos'] = df['suma_vehiculos']
        simple_data['sheet_origen'] = df['sheet_origen'].iloc[0] if 'sheet_origen' in df.columns else 'Unknown'
        
        # Información extraída del filename
        #simple_data['codigo'] = codigo
        simple_data['interseccion'] = interseccion
        #simple_data['es_atipico'] = es_atipico
        simple_data['dia_tipo'] = dia_tipo
        #simple_data['filename'] = basename
            
        # Crear DataFrame
        simple_df = pd.DataFrame(simple_data)
        all_data.append(simple_df)
    
    return all_data

Combinar datasets

In [56]:
# Retorna un dataframe combinado
def combined_dataset(datasets):
    """Retorna un dataframe combinado con suma_vehiculos por dirección (N,S,E,O)"""
    
    # Concatenar verticalmente primero
    final_df = pd.concat(datasets, axis=0, ignore_index=True)
    
    # Pivotar para tener una columna por cada dirección (sheet_origen)
    pivot_df = final_df.pivot_table(
        index=['interseccion', 'dia_tipo'], 
        columns='sheet_origen', 
        values='suma_vehiculos', 
        aggfunc='first'
    ).reset_index()
    
    # Renombrar las columnas para que tengan el formato deseado
    pivot_df.columns.name = None  # Quitar el nombre del índice de columnas
    
    # Crear nombres de columnas con el formato suma_vehiculos_X
    new_columns = {}
    for col in pivot_df.columns:
        if col in ['N', 'S', 'E', 'O']:
            new_columns[col] = f'suma_vehiculos_{col}'
        else:
            new_columns[col] = col
    
    pivot_df = pivot_df.rename(columns=new_columns)
    
    # Rellenar valores NaN con 0 para direcciones que no existen
    direction_cols = ['suma_vehiculos_N', 'suma_vehiculos_S', 'suma_vehiculos_E', 'suma_vehiculos_O']
    for col in direction_cols:
        if col not in pivot_df.columns:
            pivot_df[col] = 0
        else:
            pivot_df[col] = pivot_df[col].fillna(0)
    
    # Crear secuencia de tiempo cada 15 minutos desde 6:30
    num_rows = len(pivot_df)
    hora_inicio = []
    hora_fin = []
    
    # Empezar desde 6:30 (390 minutos desde medianoche)
    start_minutes = 6 * 60 + 30  # 6:30 = 390 minutos
    
    for i in range(num_rows):
        # Calcular minutos para hora inicio
        inicio_minutes = start_minutes + (i * 15)
        fin_minutes = inicio_minutes + 15
        
        # Convertir a horas y minutos para inicio
        inicio_hours = (inicio_minutes // 60) % 24
        inicio_mins = inicio_minutes % 60
        
        # Convertir a horas y minutos para fin
        fin_hours = (fin_minutes // 60) % 24
        fin_mins = fin_minutes % 60
        
        # Formatear como HH:MM
        hora_inicio_str = f"{inicio_hours:02d}:{inicio_mins:02d}"
        hora_fin_str = f"{fin_hours:02d}:{fin_mins:02d}"
        
        hora_inicio.append(hora_inicio_str)
        hora_fin.append(hora_fin_str)
    
    # Agregar las columnas de tiempo
    pivot_df['hora_inicio'] = hora_inicio
    pivot_df['hora_fin'] = hora_fin
    
    # Reordenar columnas
    columns_order = ['interseccion', 'dia_tipo', 'suma_vehiculos_N', 'suma_vehiculos_S', 
                    'suma_vehiculos_E', 'suma_vehiculos_O', 'hora_inicio', 'hora_fin']
    
    # Solo incluir columnas que existen
    final_columns = [col for col in columns_order if col in pivot_df.columns]
    pivot_df = pivot_df[final_columns]
    
    return pivot_df

Concatenar archivos excels

In [57]:
def extract_flujograma(file_path, names_sheet_not):
    """Extrae datos específicos entre coordenadas K42:DJ97 y calcula suma por fila"""
    try:
        all_data = []
        xls = pd.ExcelFile(file_path)
        print(f"Hojas encontradas: {xls.sheet_names}")
       
        
        for sheet_name in xls.sheet_names:
            if sheet_name in names_sheet_not:
                continue
                
            print(f"\nProcesando hoja: {sheet_name}")
            
            # Leer toda la hoja sin header
            df_raw = pd.read_excel(xls, sheet_name=sheet_name, header=None)
            print(f"Forma total: {df_raw.shape}")
  
        
            
            start_row = 41  # K42 (fila 42 en Excel)
            end_row = 96    # K97 (fila 97 en Excel)
            start_col = 10  # K (columna K en Excel)
            end_col = 113   # DJ (columna DJ en Excel - DJ es la columna 114)
                
            if len(df_raw) <= end_row or len(df_raw.columns) <= end_col:
                print(f" Dimensiones insuficientes. Necesario: filas>{end_row}, cols>{end_col}")
                print(f"  Actual: filas={len(df_raw)}, cols={len(df_raw.columns)}")
                continue
            
            # Extraer la región K42:DJ97
            data_region = df_raw.iloc[start_row:end_row+1, start_col:end_col+1].copy()
            print(f"Región extraída: {data_region.shape}")
            
            # Procesar cada fila y calcular suma
            clean_data = []
            
            for i, row_idx in enumerate(range(start_row, end_row+1)):
                excel_row = row_idx + 1  # Convertir a número de fila de Excel
                
                # Obtener la fila completa original para contexto (hora, etc.)
                full_row = df_raw.iloc[row_idx]
                
                # Obtener la región de datos numéricos (K:DJ)
                numeric_region = df_raw.iloc[row_idx, start_col:end_col+1]
                
                # Convertir a numérico y calcular suma
                numeric_values = pd.to_numeric(numeric_region, errors='coerce').fillna(0)
                row_sum = numeric_values.sum()
                
                # Extraer información contextual (primeras columnas)
                hora = str(full_row[0]) if pd.notna(full_row[0]) else ""
                
                # Solo incluir filas que tengan datos válidos
                if row_sum > 0 :
                    clean_data.append({
                        'suma_vehiculos': row_sum,
                        'sheet_origen': sheet_name
                    })
                    
                    # Mostrar algunas filas de ejemplo
                    if i < 3:
                        print(f"  Fila {excel_row}: hora='{hora}', suma={row_sum}")
            if clean_data:
                df_clean = pd.DataFrame(clean_data)
                all_data.append(df_clean)
                print(f" Extraídas {len(clean_data)} filas válidas")
                print(f" Suma total de vehículos: {df_clean['suma_vehiculos'].sum()}")
            else:
                print(f" No se encontraron datos válidos")
        
    

        return all_data
    
    except Exception as e:
        print(f"Error procesando archivo: {e}")
        return None
            
            
            

In [58]:
def flujograma_concat_excels(path_files: str, name_file: str, list_not):
    """Une todos los archivos Excel de una carpeta, limpia columnas y combina sheets"""
    files = glob.glob(path_files)
    print(f"Archivos encontrados: {len(files)}")
    print("Archivos:", files)
    
    all_combined_data = []

    for file in files:

        print(f"\nProcesando archivo: {file}")
        
        # Usar la función separate_excel_sheets para cada archivo
        result_df_list = extract_flujograma(file, list_not)
        results_simple_dfs = simple_dataset_flujograma(result_df_list, file )
        final_df = combined_dataset(results_simple_dfs)
            
        all_combined_data.append(final_df)
        # print(f"  - Total filas del archivo: {len(file_combined)}")

    # Combinar todos los archivos
    if all_combined_data:
        final_data = pd.concat(all_combined_data, ignore_index=True)
        
        # Guardar como CSV
        final_data.to_csv(name_file, index=False)
        
        print(f"\nDataset final guardado: {name_file}")
        print(f" Total archivos procesados: {len(files)}")
        print(f" Total filas: {len(final_data)}")
        print(f" Total columnas: {len(final_data.columns)}")
        
        return final_data
    else:
        print("No se pudieron procesar los archivos")
        return None

In [42]:
# Borrador

# Borrador - Verificar coordenadas primero
excel_f = "/home/jazmin/Escritorio/AlertaVIAl/BD_InnovaThon/Flujograma_intersecciones/58_SS77-01_Av. Rosa Lozano-Jr. Geranios_Día_Atípico..xlsx"
list_not = ["Inicio", 'V_Ma', "V_Md", 'Ciclo Semaforico', 'Para Vissim']


In [59]:
datas = flujograma_concat_excels("../../BD_InnovaThon/Flujograma_intersecciones/*.xlsx", "dataset_flujograma.xlsx", list_not)
print(datas)

Archivos encontrados: 28
Archivos: ['../../BD_InnovaThon/Flujograma_intersecciones/61_SS34-01_Av. Los Próceres - Ingreso A La Cruceta_Día_Atípico.xlsx', '../../BD_InnovaThon/Flujograma_intersecciones/58_SS77-01_Av. Rosa Lozano-Jr. Geranios_Día_Típico..xlsx', '../../BD_InnovaThon/Flujograma_intersecciones/SP08 Av. Pachacutec - Av. Pardo_Día_Típico.xlsx', '../../BD_InnovaThon/Flujograma_intersecciones/SP09 Av. Pachacutec -  Av. 26 de Noviembre _Día_Atípico.xlsx', '../../BD_InnovaThon/Flujograma_intersecciones/58_SS77-01_Av. Rosa Lozano-Jr. Geranios_Día_Atípico..xlsx', '../../BD_InnovaThon/Flujograma_intersecciones/SP02 Av. Los Héroes - Ca. Buckinghan_Día_Tipico.xlsx', '../../BD_InnovaThon/Flujograma_intersecciones/SP06 Av. Pachacutec - Jr. Huamachuco - Av. 1_Día_Típico.xlsx', '../../BD_InnovaThon/Flujograma_intersecciones/SP07 Av. Pachacutec -  8 de Octubre - Huayna Capac_Día_Típico.xlsx', '../../BD_InnovaThon/Flujograma_intersecciones/SP01 Av. Los Héroes - Ca. Los Conquistadores_Día_Tip